In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval import PyMCConfig, run_evaluation

# Load your data

In [ ]:
# Load your data - we have a revenue column and a response column (quantity)
data = pd.read_csv("data/example_data.csv")
data.head()

# Fit a PyMC MMM

In [ ]:
# Fit a pymc model

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

base_model = MMM(
    date_column="date_week" ,
    channel_columns=["TV","radio"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

base_model.fit(X=X, y=y, chains=4, target_accept=0.85)

# I wonder if this MMM is any good... let's check!

In [ ]:
# We just need to make a config using the model object and some fit kwargs

fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

config = PyMCConfig.from_model_object(base_model, fit_kwargs=fit_kwargs, response_column="quantity", revenue_column="revenue")

# Save this for later if you want to run from CLI!
config.save_model_object_to_json(save_path="data/", file_name="saved_config")

In [ ]:
# Run the evaluation suite!
result = run_evaluation(framework="pymc_marketing", config=config, data=data)

# Examine the results

In [ ]:
# Let's see what we got
display(result)